In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
save_time = 5

In [ ]:
left_x = -300
right_x = 250

m_1 = 1
m_2 = 0.5
stiffness = 1
d_1 = 0
d_2 = 0
omega = 1
a = 1

num = np.round(np.arange(left_x, right_x, a)/a)
mass = np.array([m_1 if i < 0 else m_2 for i in num])
base_stiffness = np.array([d_1 if j < 0 else d_2 for j in num])

In [ ]:
print(f'Max Omega Chain 1: {2*np.sqrt(stiffness/m_1)}')
print(f'Max Omega Chain 2: {2*np.sqrt(stiffness/m_2)}')

In [ ]:
def energy(mass, c, vel, disp):
    e = mass/2 * vel**2 + c/4 * (np.roll(disp,-1)+np.roll(disp,1)-2*disp-base_stiffness*disp)**2
    return e

In [ ]:
def specify_initial_and_boundary(num, beta, n_0, u_0, stiffness, m_1, a, omega):

    k_1 = np.arcsin(np.sqrt(m_1*(omega**2-d_1/m_1)/(4*stiffness))) * 2 / a
    g_1 = a/(2*omega)*np.sqrt((omega**2-d_1/m_1)*((4*stiffness+d_1)/m_1-omega**2))
    
    disp = np.array([0]*len(num), dtype=float)
    vel = np.array([0]*len(num), dtype=float)

    for i, num in enumerate(num):
        if num < 0:
            disp[i] = u_0 * np.exp(-beta**2/2 * (num - n_0)**2) * np.sin(num * a * k_1)
            vel[i] = -u_0 * np.exp(-beta**2/2 * (num - n_0)**2)
            vel[i] *= (omega * np.cos(k_1*a*num) - beta**2*g_1/a*(num-n_0)*np.sin(num * a * k_1))
        
    return disp, vel

In [ ]:
def solver(mass, disp, vel, base_stiffness, disp_history, vel_history, energy_history, t_max, dt):

    times = np.arange(0, t_max, dt)

    for t in tqdm(times):
        acc1 = (stiffness/mass)*(np.roll(disp,-1)+np.roll(disp,1)-2*disp-base_stiffness*disp)
        disp += vel*dt+1/2*acc1*dt**2
        acc2 = (stiffness/mass)*(np.roll(disp,-1)+np.roll(disp,1)-2*disp-base_stiffness*disp)
        vel += 1/2*(acc1+acc2)*dt
        #deform = np.roll(disp,-1)+np.roll(disp,1)-2*disp-base_stiffness*disp
        #vel += (stiffness/mass)*deform*dt
        #disp += vel * dt
        if t % save_time == 0:
            disp_history = np.vstack((disp_history, disp))
            vel_history = np.vstack((disp_history, vel))
            energy_history = np.vstack((energy_history, energy(mass, stiffness, vel, disp)))
    
    return disp_history, vel_history, energy_history

In [ ]:
beta = 0.03
n_0 = -150
u_0 = 1

disp, vel = specify_initial_and_boundary(num, beta, n_0, u_0, stiffness, m_1, a, omega)

plt.plot(num, disp)
plt.title(f'В начальный момент времени')
plt.xlabel('Номер частицы')
plt.ylabel('Перемещение, усл. единиц')
plt.grid()
print(max(disp))

In [ ]:
dt = 0.005
t_max = 600

disp_history = disp
vel_history = vel
energy_history = energy(mass, stiffness, vel, disp)
disp_history, vel_history, energy_history = solver(mass, disp, vel, base_stiffness, disp_history,
                                                   vel_history, energy_history, t_max, dt)

plt.plot(num, disp_history[-1])
plt.title(f'В момент времени t={t_max} усл.ед.')
plt.xlabel('Номер частицы')
plt.ylabel('Перемещение, усл. единиц')
plt.grid()

In [ ]:
energy_sum = np.sum(energy_history,axis=1)
energy_left = np.sum(np.array([e[:-left_x] for e in energy_history]),axis=1)
energy_left /= max(energy_sum)
energy_right = np.sum(np.array([e[-left_x:-left_x+right_x] for e in energy_history]),axis=1)
energy_right /= max(energy_sum)
energy_sum /= max(energy_sum)
saved_times = range(0,t_max+save_time,save_time)
plt.plot(saved_times, energy_sum)
plt.title('Полная энергия системы')
plt.xlabel('Время')
plt.ylabel('Энергия')

In [ ]:
history = energy_history

In [ ]:
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig, axs = plt.subplots(nrows=2, ncols=1, layout='constrained')

line = axs[0].plot(num, history[0], c='blue', linewidth=1)[0]
line1 = axs[1].plot(saved_times[0], energy_sum[0], c='black', linewidth=1, label="Полная энергия в системе")[0]
line2 = axs[1].plot(saved_times[0], energy_left[0], c='blue', linewidth=1, label="Энергия в левой решётке")[0]
line3 = axs[1].plot(saved_times[0], energy_right[0], c='red', linewidth=1, label="Энергия в правой решётке")[0]

axs[0].grid()
axs[0].set_ylim((min(history.flatten())-0.05,1.2*max(history.flatten())))
axs[0].plot([0,0],[-1.2,1.2], c='r', linewidth=0.8)
axs[0].set_xlabel('Номер частицы')
axs[0].set_ylabel('Энергия, усл.ед.')
axs[1].grid()
en_conc = np.concatenate([energy_sum, energy_left, energy_right])
axs[1].set_ylim((0.9*min(en_conc),1.2*max(en_conc)))
axs[1].set_xlim((0,saved_times[-1]))
axs[1].set_title('Энергия в системе')
axs[1].set_xlabel('Время, усл.ед.')
axs[1].set_ylabel('Энергия, усл.ед.')
axs[1].legend(fontsize=7)

def update(frame):
    line.set_xdata(num)
    line.set_ydata(history[frame])
    line1.set_xdata(saved_times[:frame])
    line1.set_ydata(energy_sum[:frame])
    line2.set_xdata(saved_times[:frame])
    line2.set_ydata(energy_left[:frame])
    line3.set_xdata(saved_times[:frame])
    line3.set_ydata(energy_right[:frame])
    axs[0].set_title(f't={save_time*frame} \n (m1={m_1}, m2={m_2}, c={stiffness}, a={a}, omega={omega})')
    return [line, line1, line2, line3] 

anim = animation.FuncAnimation(fig, update, frames=len(history))
anim

In [ ]:
#anim.save('1.gif')